In [35]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
subset_test_path = './new_train/train_subset'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [36]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [37]:
batch_sz = 4

def train_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [45]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
        # preprocessing more ????
        inp = inp[:,0,:,:]
        out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        file1 = open("loss_steps.txt", "a")  # append mode
        file1.write(str(loss.item()) + ",")
        file1.close()
        
#       output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()))
    

In [46]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 1

for epoch in range(1, num_epoch + 1):
        train(encoder, decoder, device, val_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

KeyboardInterrupt: 

In [32]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
        _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [38]:
output = infer_absolute(test_loader, encoder, decoder)

In [39]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
69420,"[569.1348876953125, 2250.463134765625]","[569.5634765625, 2251.495361328125]","[569.4859008789062, 2252.1455078125]","[569.4866333007812, 2252.8544921875]","[569.4404907226562, 2253.501953125]","[569.4099731445312, 2254.143310546875]","[569.3798828125, 2254.77392578125]","[569.3560791015625, 2255.40380859375]","[569.33740234375, 2256.03662109375]","[569.3240966796875, 2256.676513671875]",...,"[569.4024047851562, 2264.60791015625]","[569.4114990234375, 2265.3935546875]","[569.4180297851562, 2266.181640625]","[569.4215698242188, 2266.970458984375]","[569.4218139648438, 2267.7578125]","[569.4183349609375, 2268.5419921875]","[569.410888671875, 2269.321533203125]","[569.3991088867188, 2270.094970703125]","[569.3827514648438, 2270.8603515625]","[569.3614501953125, 2271.615966796875]"
42069,"[3418.018798828125, 1889.5478515625]","[3419.313232421875, 1888.898193359375]","[3420.30078125, 1887.8602294921875]","[3420.860595703125, 1887.1685791015625]","[3421.375244140625, 1886.5740966796875]","[3422.04736328125, 1886.0665283203125]","[3422.81396484375, 1885.5904541015625]","[3423.579345703125, 1885.0941162109375]","[3424.294677734375, 1884.5634765625]","[3424.9521484375, 1884.0118408203125]",...,"[3432.265869140625, 1879.7547607421875]","[3433.06298828125, 1879.542724609375]","[3433.85595703125, 1879.3375244140625]","[3434.631591796875, 1879.13330078125]","[3435.376708984375, 1878.924072265625]","[3436.0791015625, 1878.705078125]","[3436.72802734375, 1878.4720458984375]","[3437.314697265625, 1878.2225341796875]","[3437.83349609375, 1877.95556640625]","[3438.281494140625, 1877.671875]"
64209,"[264.63311767578125, 3289.8251953125]","[264.613525390625, 3288.366943359375]","[264.6404724121094, 3287.393798828125]","[264.46502685546875, 3286.384765625]","[264.261474609375, 3285.3447265625]","[264.0484313964844, 3284.28369140625]","[263.8317565917969, 3283.202392578125]","[263.6130065917969, 3282.099609375]","[263.3922424316406, 3280.97412109375]","[263.1689147949219, 3279.8232421875]",...,"[260.6233825683594, 3265.210693359375]","[260.4253845214844, 3263.788818359375]","[260.23236083984375, 3262.3837890625]","[260.03228759765625, 3260.998291015625]","[259.80621337890625, 3259.6298828125]","[259.5291748046875, 3258.2724609375]","[259.173828125, 3256.919921875]","[258.7164306640625, 3255.57470703125]","[258.1456604003906, 3254.255615234375]","[257.4710693359375, 3253.005615234375]"
13337,"[2905.497314453125, 1491.284423828125]","[2904.52490234375, 1490.3553466796875]","[2903.96875, 1489.91064453125]","[2903.445068359375, 1489.322021484375]","[2902.91064453125, 1488.77978515625]","[2902.39013671875, 1488.248291015625]","[2901.873046875, 1487.7222900390625]","[2901.351318359375, 1487.19287109375]","[2900.81982421875, 1486.65625]","[2900.284423828125, 1486.114501953125]",...,"[2893.701416015625, 1479.583984375]","[2893.024169921875, 1478.9228515625]","[2892.332763671875, 1478.24951171875]","[2891.626220703125, 1477.5638427734375]","[2890.903564453125, 1476.8656005859375]","[2890.16259765625, 1476.1544189453125]","[2889.400390625, 1475.4295654296875]","[2888.613525390625, 1474.689697265625]","[2887.797607421875, 1473.9337158203125]","[2886.947998046875, 1473.159912109375]"


In [40]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [41]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
69420,569.134888,2250.463135,569.563477,2251.495361,569.485901,2252.145508,569.486633,2252.854492,569.440491,2253.501953,...,569.418335,2268.541992,569.410889,2269.321533,569.399109,2270.094971,569.382751,2270.860352,569.361450,2271.615967
42069,3418.018799,1889.547852,3419.313232,1888.898193,3420.300781,1887.860229,3420.860596,1887.168579,3421.375244,1886.574097,...,3436.079102,1878.705078,3436.728027,1878.472046,3437.314697,1878.222534,3437.833496,1877.955566,3438.281494,1877.671875
64209,264.633118,3289.825195,264.613525,3288.366943,264.640472,3287.393799,264.465027,3286.384766,264.261475,3285.344727,...,259.529175,3258.272461,259.173828,3256.919922,258.716431,3255.574707,258.145660,3254.255615,257.471069,3253.005615
13337,2905.497314,1491.284424,2904.524902,1490.355347,2903.968750,1489.910645,2903.445068,1489.322021,2902.910645,1488.779785,...,2890.162598,1476.154419,2889.400391,1475.429565,2888.613525,1474.689697,2887.797607,1473.933716,2886.947998,1473.159912


In [42]:
df2.to_csv("outputs1ep_train.csv", index=True)